In [472]:
reload_lamb()

In [473]:
import contextlib
from lamb.utils import ensuremath

class MaybeElem:
    def __repr__(self):
        return 'Maybe'

    def __eq__(self, other):
        return isinstance(types.demeta(other), MaybeElem)

    def __hash__(self):
        return hash("MaybeElem")

    def __mt_key__(self):
        return ('  Maybe',)

class MaybeSet(types.ComplexDomainSet):
    def __init__(self, typ):
        finite = isinstance(typ.arg.domain, types.DomainSet) and typ.arg.domain.finite
        super().__init__("M", typ, finite=finite)

    def check(self,x):
        # this is the tricky bit: a meta element is a member if it is MaybeElem, or
        # if it is a member of the wrapped type's domain. So e.g. this allows `True`
        # to be present in the domain for type `Mt`. Successive `M` type operators
        # recurse, but `MaybeElem` is only present once at any type because this
        # check short-circuits the recursion. See below for examples of the sets
        # this produces.
        return (isinstance(types.demeta(x), MaybeElem)
                or isinstance(self.type.arg.domain, types.DomainSet) and self.type.arg.domain.check(x))

    def __len__(self):
        if not self.finite:
            raise ValueError("Non-finite `MaybeDomainSet`s do not have a length.")
        return len(self.type.arg.domain) + 1

    def __iter__(self):
        if not self.finite:
            raise ValueError("Can't iterate over non-finite `MaybeDomainSet`s.")
        if MaybeElem() not in self.type.arg.domain:
            yield MaybeElem()
        yield from iter(self.type.arg.domain)
    
    def enumerable(self):
        # only the finite case is supported right now...
        return self.finite

    def normalize(self, x):
        if x in self.type.arg.domain:
            return self.type.arg.domain.normalize(x)
        else:
            return MaybeElem()

    def element_repr(self, x, rich=False, **kwargs):
        if isinstance(types.demeta(x), MaybeElem):
            return "Maybe"
        else:
            return self.type.arg.domain.element_repr(x, rich=rich)

    def __repr__(self):
        return f"MaybeSet({self.type})"

    @classmethod
    def element_to_type(cls, x, ctrl=None):
        raise lamb.parsing.ParseError("`Maybe` terms require a complex type")


class MaybeType(types.TypeConstructor):
    domain_class = MaybeSet
    def __init__(self, c):
        super().__init__(c, symbol="M")
        
    def __repr__(self):
        return f"M{repr(self.arg)}"

    def functional(self):
        return self.arg.functional()

    @property
    def left(self):
        return self.arg.left

    @property
    def right(self):
        return self.arg.right

    @property
    def arg(self):
        return self.signature[0]
    
    @classmethod
    def parse(cls, s, i, parse_control_fun):
        next = lamb.parsing.consume_char(s, i, "M")
        if next is None:
            return (None, i)
        else:
            i = next
            (arg, i) = parse_control_fun(s, i)
            return (MaybeType(arg), i)

    def latex_str(self):
        return utils.ensuremath(f"M{self.arg.latex_str()}")

class MaybeExpr(meta.TypedExpr):
    """Given some expression of type `X`, wrap it (leaving the value
    unchanged) so that it is type `MX`."""
    def __init__(self, arg, typ=None, **kwargs):
        typ, _ = self.type_constraint(MaybeType(arg.type), typ)
        super().__init__("MaybeWrap", arg, typ=typ)

    def simplify(self, **sopts):
        if self[0].meta():
            return meta.MetaTerm(self[0].op, typ=self.type)
        else:
            return self
    
    def __repr__(self):
        return f"MaybeWrap({repr(self[0])})"

    def latex_str(self, suppress_parens=False, **kwargs):
        body_str = self[0].latex_str(suppress_parens=True, **kwargs)
        return utils.ensuremath(f"\\textsf{{MaybeWrap}}({body_str})")

# class MaybeTerm(meta.TypedTerm):
#     def __init__(self, copying=False):
#         super().__init__("Maybe", typ=MaybeType(types.VariableType.any()))

#     # def meta(self):
#     #     return True

#     def should_show_type(self, assignment=None):
#         return True
    
#     def simplify(self, **sopts):
#         if not self.type.polymorphic():
#             return meta.MetaTerm(MaybeElem(), typ=self.type)
#         return self

class MaybeTerm(meta.TypedExpr):
    term_style = True
    def __init__(self, typ=None, **kwargs):
        typ, _ = self.type_constraint(MaybeType(types.VariableType.any()), typ)
        super().__init__("Maybe", typ=typ)

    def should_show_type(self, assignment=None):
        return True

    def term(self):
        return True
    
    def simplify(self, **sopts):
        if not self.type.is_polymorphic():
            return meta.MetaTerm(MaybeElem(), typ=self.type)
        return self

    def latex_str(self, **kwargs):
        return ensuremath(f"\\text{{Maybe}}_{{{self.type.latex_str()}}}")

class UnmaybeExpr(meta.TypedExpr):
    """Given some maybe-typed expression, simplify to an expression of the inner type
    (producing a DomainError if the value is Maybe)"""
    def __init__(self, arg, typ=None, **kwargs):
        self.type_constraint(arg, MaybeType)
        typ, _ = self.type_constraint(arg.type.arg, typ) # XX not general enough?
        super().__init__("Unmaybe", arg, typ=typ)

    def simplify(self, **sopts):
        if isinstance(self[0], MaybeExpr):
            return self[0][0]
        elif self[0].meta():
            if isinstance(self[0].op, MaybeElem):
                raise meta.meta.DomainError(self[0], self.type.domain, "`Unmaybe(Maybe)` is undefined")
            return meta.MetaTerm(self[0].op, typ=self.type)
        else:
            return self
    
    def latex_str(self, suppress_parens=False, **kwargs):
        # override in order to show `Unmaybe` with sans serif
        body_str = self[0].latex_str(suppress_parens=True, **kwargs)
        return utils.ensuremath(f"\\textsf{{Unmaybe}}({body_str})")





Maybe = MaybeElem()

meta.core.reset_type_system()
# meta.reset()
# system = lang.td_system.copy()
meta.get_type_system().add_nonatomic(MaybeType)
# type_m = types.BasicType('m', MaybeSet())
# system.add_basic_type(type_m)
# lang.set_system(system)

meta.global_namespace()['Maybe'] = MaybeTerm()
# meta.TypedExpr.add_local('Maybe', MaybeTerm)

# lang.get_system().lexicon['Maybe'] = MaybeTerm()
# meta.TypedExpr.add_local('Maybe', MaybeTerm())
# meta.TypedExpr.add_local('Undef', lambda x: x.equivalent_to(MaybeTerm()))
meta.TypedExpr.add_local('MaybeWrap', MaybeExpr)
meta.TypedExpr.add_local('Unmaybe', UnmaybeExpr)
# meta.TypedExpr.add_local('Case', Case.from_tuple)



In [474]:
from lamb.meta.core import SyncatOpExpr, registry

def is_maybe(x):
    return x.meta() and isinstance(x.op, MaybeElem)

class MaybeEqExpr(SyncatOpExpr):
    canonical_name = "<=>"
    secondary_names = {"==", "%"}
    op_name_latex = "="
    commutative = True
    associative = True

    arg_signature = tp("(MX,MX)")

    def __init__(self, arg1, arg2, typ=None, **kwargs):
        typ = self.type_constraint(typ, types.type_t, constant=True)
        super().__init__(typ, arg1, arg2)

    def _compile(self):
        l = self[0]._compiled
        r = self[1]._compiled
        return lambda context: l(context) == r(context)

    def simplify(self, **sopts):
        if is_maybe(self[0]):
            if is_maybe(self[1]):
                return derived(true_term, self, desc="Maybe identity")
            elif isinstance(self[1], MaybeExpr):
                return derived(false_term, self, desc="Maybe identity")
            elif isinstance(self[1], lamb.meta.boolean.Case):
                # this is very special case-y, but helpful for some cases in this notebook
                return derived(
                    self[1].copy_local(self[1][0],
                                          self[1][1].equivalent_to(self[0]),
                                          self[1][2].equivalent_to(self[0])),
                    self,
                    "distribute over Case")
        elif is_maybe(self[1]):
            if isinstance(self[0], MaybeExpr):
                return derived(false_term, self, desc="Maybe identity")
            elif isinstance(self[0], lamb.meta.boolean.Case):
                return derived(
                    self[0].copy_local(self[0][0],
                                          self[0][1].equivalent_to(self[1]),
                                          self[0][2].equivalent_to(self[1])),
                    self,
                    "distribute over Case")
        return self

registry.add_operator(MaybeEqExpr)

In [475]:
te("Maybe")

Maybe_M∀X

In [476]:
x = %te Maybe
x

Maybe_M∀X

In [414]:
repr(x)

'Maybe'

In [415]:
te(MaybeTerm())

Maybe

In [416]:
# meta.core.reset_type_system()
meta.get_type_system().type_ranking

{lamb.types.BasicType: 0,
 lamb.types.FunType: 0,
 lamb.types.SetType: 0,
 lamb.types.TupleType: 0,
 lamb.types.UnknownType: 10,
 lamb.types.VariableType: 10,
 lamb.types.Forall: 20,
 lamb.types.DisjunctiveType: 1,
 __main__.MaybeType: 0}

In [417]:
from lamb.meta.boolean import Case
Case(te("p_t"), te('_c1'), te('_c3'))

Case(p_t, _c1, _c3)

In [418]:
Case(te("p_t"), te('_c1'), Case(te('p1_t'), te('_c2'), te('_c3')))

Case(p_t, _c1, Case(p1_t, _c2, _c3))

In [419]:

%te simplify Case(True, _c1, _c2)

_c1

In [420]:
MaybeExpr(te("x_e"))

MaybeWrap(x_e)

In [421]:
te("MaybeWrap(x_t)").type

Mt

In [422]:
%te L x_e : MaybeWrap(x)

(λ x_e: MaybeWrap(x_e))

In [423]:
UnmaybeExpr(MaybeExpr(te("x_e")))

Unmaybe(MaybeWrap(x_e))

In [353]:
UnmaybeExpr(MaybeExpr(te("x_e"))).simplify_all()

x_e

In [354]:
%te L x_t : Unmaybe(MaybeWrap(x))

(λ x_t: Unmaybe(MaybeWrap(x_t)))

In [355]:
%te simplify L x_t : Unmaybe(MaybeWrap(x))

(λ x_t: x_t)

In [356]:
%te simplify (L x_Mt : Unmaybe(x))(True_Mt)

True

In [357]:
%te simplify (L x_Mt : Unmaybe(x))(Maybe)

Unmaybe(Maybe)

In [358]:
meta.MetaTerm(Maybe, typ=tp("Mt"))

Maybe

In [359]:
%te Maybe_Mt

Maybe

In [360]:
%te simplify Maybe_Mt

Maybe

In [361]:
with lamb.errors():
    meta.MetaTerm(Maybe)

<span style="color:red">**ParseError**</span>: `Maybe` terms require a complex type

In [362]:
meta.MetaTerm(Maybe, typ=tp("MMt"))

Maybe

In [363]:
meta.MetaTerm(Maybe, typ=tp("MMt")) == meta.MetaTerm(Maybe, typ=tp("Mt"))

False

In [364]:
meta.MetaTerm(Maybe, typ=tp("MMt")).op == meta.MetaTerm(Maybe, typ=tp("Mt")).op

True

In [365]:
meta.MetaTerm(Maybe, typ=tp("MMt")).equivalent_to(meta.MetaTerm(Maybe, typ=tp("MMt"))).simplify_all()

True

In [366]:
meta.MetaTerm(Maybe, typ=tp("MMt")).equivalent_to(meta.MetaTerm(Maybe, typ=tp("Mt")))

ParseError: No viable 2-ary operator for symbol `<=>` and arguments `(Maybe, Maybe)`

In [431]:
%te Maybe_MMt == MaybeWrap(Maybe_Mt)

(Maybe <=> MaybeWrap(Maybe))

In [432]:
%te simplify Maybe_MMt == MaybeWrap(Maybe_Mt)

True

In [367]:
%te Maybe

Maybe

In [368]:
%te True_Mt

True

In [369]:
s = te("Set x_Mt : True").eliminate()
s

{False, True, Maybe}

In [370]:
s = te("Set x_MMt : True").eliminate()
s

{False, True, Maybe}

In [371]:
lamb.parsing.errors_raise=False

In [372]:
%%lamb
# Undef = L x_X : x <=> Maybe
unit = L x_X : MaybeWrap(x_X)
bind = L m_MX : L k_<X,MY> : Case(m <=> Maybe, Maybe, k(Unmaybe(m)))


${unit}_{\left\langle{}X,MX\right\rangle{}}\:=\:\lambda{} x_{X} \: . \: \textsf{MaybeWrap}({x})$<br />
${bind}_{\left\langle{}MX,\left\langle{}\left\langle{}X,MY\right\rangle{},MY\right\rangle{}\right\rangle{}}\:=\:\lambda{} m_{MX} \: . \: \lambda{} k_{\left\langle{}X,MY\right\rangle{}} \: . \: \begin{cases}
\text{Maybe}_{MY} & \text{if }{m} = \text{Maybe}_{MX} \\{k}(\textsf{Unmaybe}({m})) & \text{otherwise} \end{cases}$

In [373]:
%lamb liftedapply = L f_M<X,Y> : L x_MX : bind(f)(L a_<X,Y> : bind(x)(L b_X : unit(a(b))))


${liftedapply}_{\left\langle{}M\left\langle{}X,X'\right\rangle{},\left\langle{}MX,MX'\right\rangle{}\right\rangle{}}\:=\:\lambda{} f_{M\left\langle{}X,X'\right\rangle{}} \: . \: \lambda{} x_{MX} \: . \: \begin{cases}
\text{Maybe}_{MX'} & \text{if }{f} = \text{Maybe}_{M\left\langle{}X,X'\right\rangle{}} \\
\text{Maybe}_{MX'} & \text{if }{x} = \text{Maybe}_{MX} \\\textsf{MaybeWrap}(\textsf{Unmaybe}({f})(\textsf{Unmaybe}({x}))) & \text{otherwise} \end{cases}$

In [374]:
%lamb lapply = (L m_M∀X : L f_M<X,Y> : L x_MX : Case(x <=> Maybe, Maybe, Case(f <=> Maybe, Maybe, MaybeWrap((Unmaybe(f))(Unmaybe(x))))))


${lapply}_{\left\langle{}M\forall{}X,\left\langle{}M\left\langle{}X,Y\right\rangle{},\left\langle{}MX,MY\right\rangle{}\right\rangle{}\right\rangle{}}\:=\:\lambda{} m_{M\forall{}X} \: . \: \lambda{} f_{M\left\langle{}X,Y\right\rangle{}} \: . \: \lambda{} x_{MX} \: . \: \begin{cases}
\text{Maybe}_{MY} & \text{if }{x} = \text{Maybe}_{MX} \\
\text{Maybe}_{MY} & \text{if }{f} = \text{Maybe}_{M\left\langle{}X,Y\right\rangle{}} \\\textsf{MaybeWrap}(\textsf{Unmaybe}({f})(\textsf{Unmaybe}({x}))) & \text{otherwise} \end{cases}$

In [375]:
bind(unit(te(True))).simplify_all(reduce=True)

(λ k_<t,MY>: k_<t,MY>(True))

In [376]:
meta.core.let_wrapper(bind(MaybeTerm()).simplify_all(reduce=True))

(λ k_<X,MX1>: Maybe)

In [377]:
system = lang.td_system.copy()
lang.set_system(system)
system.add_typeshift(unit, "UnitShift")
system.add_unary_rule(unit, "Unit")

system.add_binary_rule(bind, "Bind")
# system.add_binary_rule(lapply, "LiftedFA")
# do a bit of cleanup, we don't need PM here.
system.remove_rule('PM')
system.typeshift = True
system

Composition system: Type-driven composition (copy)

In [378]:
%te L x_t : Case(p_t, f(x), MaybeWrap(g(x)))

INFO (core): Coerced guessed type for 'g_e' into <t,e>, to match argument 'x_t'
INFO (core): Coerced guessed type for 'f_e' into <t,e>, to match argument 'x_t'


<span style="color:red">**ParseError**</span>: No viable 3-ary operator for symbol `Case` and arguments `(p_t, f_<t,e>(x_t), MaybeWrap(g_<t,e>(x_t)))`, in string `L x_t : Case(p_t, f(x), MaybeWrap(g(x)))`

In [395]:
%te L f_<e,t> : Case((ExistsExact x_e : f(x) & Have(T_e, x)), MaybeWrap(Iota x_e : f(x) & Have(T_e, x)), Maybe)

INFO (core): Coerced guessed type for 'Have_t' into <(e,e),t>, to match argument '(T_e, x_e)'
INFO (core): Coerced guessed type for 'Have_t' into <(e,e),t>, to match argument '(T_e, x_e)'


(λ f_<e,t>: Case((ExistsExact x_e: (f_<e,t>(x_e) & Have_<(e,e),t>(T_e, x_e))), MaybeWrap((ι x_e: (f_<e,t>(x_e) & Have_<(e,e),t>(T_e, x_e)))), Maybe))

In [396]:
%te L p_Mt : L q_Mt : Case(p <=> Maybe, Maybe, Case(p <=> MaybeWrap(False), MaybeWrap(True), q))
# %te globals L p_Mt : L q_Mt : Case(p <=> Maybe, Maybe, q)

(λ p_Mt: (λ q_Mt: Case((p_Mt <=> Maybe), Maybe, Case((p_Mt <=> MaybeWrap(False)), MaybeWrap(True), q_Mt))))

In [397]:
%%lamb
||his|| = L f_<e,t> : Case((ExistsExact x_e : f(x) & Have(T_e, x)), MaybeWrap(Iota x_e : f(x) & Have(T_e, x)), Maybe)
||suit|| = L x_e : Suit(x)
||bring|| = L y_e : L x_e : Bring(x,y)
||theo|| = Theo_e
||has_bro|| = L x_e : Bro(x)
||if_|| = L p_Mt : L q_Mt : Case(p <=> Maybe, Maybe, Case(p <=> MaybeWrap(False), MaybeWrap(True), q))

INFO (core): Coerced guessed type for 'Have_t' into <(e,e),t>, to match argument '(T_e, x_e)'
INFO (core): Coerced guessed type for 'Have_t' into <(e,e),t>, to match argument '(T_e, x_e)'
INFO (core): Coerced guessed type for 'Suit_t' into <e,t>, to match argument 'x_e'
INFO (core): Coerced guessed type for 'Bring_t' into <(e,e),t>, to match argument '(x_e, y_e)'
INFO (core): Coerced guessed type for 'Bro_t' into <e,t>, to match argument 'x_e'


$[\![\text{\textbf{his}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},Me\right\rangle{}} \:=\: \lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {f}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {f}({x}) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases}$<br />
$[\![\text{\textbf{suit}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: \lambda{} x_{e} \: . \: {Suit}({x})$<br />
$[\![\text{\textbf{bring}}]\!]^{}_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \:=\: \lambda{} y_{e} \: . \: \lambda{} x_{e} \: . \: {Bring}({x}, {y})$<br />
$[\![\text{\textbf{theo}}]\!]^{}_{e} \:=\: {Theo}_{e}$<br />
$[\![\text{\textbf{has\_bro}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: \lambda{} x_{e} \: . \: {Bro}({x})$<br />
$[\![\text{\textbf{if\_}}]\!]^{}_{\left\langle{}Mt,\left\langle{}Mt,Mt\right\rangle{}\right\rangle{}} \:=\: \lambda{} p_{Mt} \: . \: \lambda{} q_{Mt} \: . \: \begin{cases}
\textsf{Maybe}_{Mt} & \text{if }{p} = \textsf{Maybe}_{Mt} \\
\textsf{True}_{Mt} & \text{if }{p} = \textsf{False}_{Mt} \\{q} & \text{otherwise} \end{cases}$

In [398]:
bring

In [399]:
his * suit

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[his suit]}}]\!]^{}_{Me} \:=\: \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases}$

In [400]:
((lang.Binder(1) * ((theo * (bring * lang.Trace(1))) * None)) * (theo * None)).tree()

1 composition path:<br /><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:block;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div style="display:table;table-layout:auto;" align="center"><div style="display:table-row;"><div style="display:table-cell;padding-right:2px; padding-left:2px;" align="center"><div style="display:inline-block;"><span>$[\![\text{\textbf{1}}]\!]^{}$</span></div></div></div><div style="display:table-row;"><div style="display:table-cell;padding-right:2px; padding-left:2px;" align="center"><div style="display:inline-block;"><span>[idx: 1]</span></div></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1px;">*</span></div><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:block;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div style="display:table;table-layout:auto;" align="center"><div style="display:table-row;"><div style="display:table-cell;padding-right:2px; padding-left:2px;" align="center"><div style="display:inline-block;"><span>$[\![\text{\textbf{theo}}]\!]^{}_{e}$</span></div></div></div><div style="display:table-row;"><div style="display:table-cell;padding-right:2px; padding-left:2px;" align="center"><div style="display:inline-block;"><span>${Theo}_{e}$</span></div></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1px;">*</span></div><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:block;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div style="display:table;table-layout:auto;" align="center"><div style="display:table-row;"><div style="display:table-cell;padding-right:2px; padding-left:2px;" align="center"><div style="display:inline-block;

In [401]:
bind

(λ m_MX: (λ k_<X,MY>: Case((m_MX <=> Maybe), Maybe, k_<X,MY>(Unmaybe(m_MX)))))

In [402]:
(his * suit) * (lang.Binder(1) * ((theo * (bring * lang.Trace(1))) * None))

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[[his suit] [1 [[theo [bring t1]]]]]}}]\!]^{}_{Mt} \:=\: \begin{cases}
\textsf{Maybe}_{Mt} & \text{if }\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }(\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases} = \textsf{Maybe}_{Me} \\\textsf{MaybeWrap}({Bring}({Theo}_{e}, \textsf{Unmaybe}(\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases}))) & \text{otherwise} \end{cases}$

In [403]:
if_ * ((theo * has_bro) * None)

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[if\_ [[theo has\_bro]]]}}]\!]^{}_{\left\langle{}Mt,Mt\right\rangle{}} \:=\: \lambda{} q_{Mt} \: . \: \begin{cases}
\textsf{Maybe}_{Mt} & \text{if }\textsf{Maybe}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
\textsf{True}_{Mt} & \text{if }\textsf{False}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\{q} & \text{otherwise} \end{cases}$

In [404]:
if_ * ((theo * has_bro) * None) * ((his * suit) * (lang.Binder(1) * ((theo * (bring * lang.Trace(1))) * None)))

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[[if\_ [[theo has\_bro]]] [[his suit] [1 [[theo [bring t1]]]]]]}}]\!]^{}_{Mt} \:=\: \begin{cases}
\textsf{Maybe}_{Mt} & \text{if }\textsf{Maybe}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
\textsf{True}_{Mt} & \text{if }\textsf{False}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
\textsf{Maybe}_{Mt} & \text{if }\textsf{Maybe}_{Me} = \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }(\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases} \\\textsf{MaybeWrap}({Bring}({Theo}_{e}, \textsf{Unmaybe}(\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases}))) & \text{otherwise} \end{cases}$

Observation: we can get the local reading by scoping out at a lower type??

In [405]:
lang.Binder(2) * ((if_ * ((theo * has_bro) * None)) * (lang.Trace(2, typ=tp('Mt'))))

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[2 [[if\_ [[theo has\_bro]]] t2]]}}]\!]^{}_{\left\langle{}Mt,Mt\right\rangle{}} \:=\: \lambda{} x_{Mt} \: . \: \begin{cases}
\textsf{Maybe}_{Mt} & \text{if }\textsf{Maybe}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
\textsf{True}_{Mt} & \text{if }\textsf{False}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\{x} & \text{otherwise} \end{cases}$

In [406]:
((his * suit) * (lang.Binder(1) * ((theo * (bring * lang.Trace(1))) * None))) * (lang.Binder(2) * ((if_ * ((theo * has_bro) * None)) * (lang.Trace(2, typ=tp('Mt')))))

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[[[his suit] [1 [[theo [bring t1]]]]] [2 [[if\_ [[theo has\_bro]]] t2]]]}}]\!]^{}_{Mt} \:=\: \begin{cases}
\textsf{Maybe}_{Mt} & \text{if }\textsf{Maybe}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
\textsf{True}_{Mt} & \text{if }\textsf{False}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
\textsf{Maybe}_{Mt} & \text{if }\textsf{Maybe}_{Me} = \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }(\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases} \\\textsf{MaybeWrap}({Bring}({Theo}_{e}, \textsf{Unmaybe}(\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases}))) & \text{otherwise} \end{cases}$

In [407]:
((his * suit) * (lang.Binder(1) * ((theo * (bring * lang.Trace(1))) * None * None)))

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[[his suit] [1 [[[theo [bring t1]]]]]]}}]\!]^{}_{MMt} \:=\: \begin{cases}
\textsf{Maybe}_{MMt} & \text{if }\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }(\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases} = \textsf{Maybe}_{Me} \\\textsf{MaybeWrap}(\textsf{MaybeWrap}({Bring}({Theo}_{e}, \textsf{Unmaybe}(\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases})))) & \text{otherwise} \end{cases}$

In [408]:
g_reading = ((his * suit) * (lang.Binder(1) * ((theo * (bring * lang.Trace(1))) * None * None))) * (lang.Binder(2) * ((if_ * ((theo * has_bro) * None)) * (lang.Trace(2, typ=tp('Mt')))))
g_reading

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[[[his suit] [1 [[[theo [bring t1]]]]]] [2 [[if\_ [[theo has\_bro]]] t2]]]}}]\!]^{}_{Mt} \:=\: \begin{cases}
\textsf{Maybe}_{Mt} & \text{if }\begin{cases}
\textsf{Maybe}_{MMt} & \text{if }(\textsf{Maybe}_{Me} = \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }(\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases}) \\\textsf{MaybeWrap}(\textsf{MaybeWrap}({Bring}({Theo}_{e}, \textsf{Unmaybe}(\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases})))) & \text{otherwise} \end{cases} = \textsf{Maybe}_{MMt} \\
\textsf{Maybe}_{Mt} & \text{if }\textsf{Maybe}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
\textsf{True}_{Mt} & \text{if }\textsf{False}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\\textsf{Unmaybe}(\begin{cases}
\textsf{Maybe}_{MMt} & \text{if }\textsf{Maybe}_{Me} = \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }(\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases} \\\textsf{MaybeWrap}(\textsf{MaybeWrap}({Bring}({Theo}_{e}, \textsf{Unmaybe}(\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\\textsf{Maybe}_{Me} & \text{otherwise} \end{cases})))) & \text{otherwise} \end{cases}) & \text{otherwise} \end{cases}$

In [409]:
g_reading[0].content

Case((Case((Maybe <=> Case((ExistsExact x_e: (Have_<(e,e),t>(T_e, x_e) & Suit_<e,t>(x_e))), MaybeWrap((ι x_e: (Have_<(e,e),t>(T_e, x_e) & Suit_<e,t>(x_e)))), Maybe)), Maybe, MaybeWrap(MaybeWrap(Bring_<(e,e),t>(Theo_e, Unmaybe(Case((ExistsExact x_e: (Have_<(e,e),t>(T_e, x_e) & Suit_<e,t>(x_e))), MaybeWrap((ι x_e: (Have_<(e,e),t>(T_e, x_e) & Suit_<e,t>(x_e)))), Maybe)))))) <=> Maybe), Maybe, Case((Maybe <=> MaybeWrap(Bro_<e,t>(Theo_e))), Maybe, Case((False <=> MaybeWrap(Bro_<e,t>(Theo_e))), True, Unmaybe(Case((Maybe <=> Case((ExistsExact x_e: (Have_<(e,e),t>(T_e, x_e) & Suit_<e,t>(x_e))), MaybeWrap((ι x_e: (Have_<(e,e),t>(T_e, x_e) & Suit_<e,t>(x_e)))), Maybe)), Maybe, MaybeWrap(MaybeWrap(Bring_<(e,e),t>(Theo_e, Unmaybe(Case((ExistsExact x_e: (Have_<(e,e),t>(T_e, x_e) & Suit_<e,t>(x_e))), MaybeWrap((ι x_e: (Have_<(e,e),t>(T_e, x_e) & Suit_<e,t>(x_e)))), Maybe))))))))))